In [5]:
!pip install pandas numpy scikit-learn joblib nltk



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
import joblib
import os
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [25]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from utils.text_preprocessing import preprocess_text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [16]:
import pandas as pd
import os

csv_path = r"C:\Users\Anjal\OneDrive\Desktop\starbucks_reviews\archive (3)\reviews_data.csv"

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ File not found: {csv_path}")

df = pd.read_csv(csv_path)
print("✅ CSV loaded successfully!")
df.head()


✅ CSV loaded successfully!


,name,location,Date,Rating,Review,Image_Links
0,Helen,"Wichita Falls, TX","Reviewed Sept. 13, 2023",5.0,Amber and LaDonna at the Starbucks on Southwes...,['No Images']
1,Courtney,"Apopka, FL","Reviewed July 16, 2023",5.0,** at the Starbucks by the fire station on 436...,['No Images']
2,Daynelle,"Cranberry Twp, PA","Reviewed July 5, 2023",5.0,I just wanted to go out of my way to recognize...,['https://media.consumeraffairs.com/files/cach...
3,Taylor,"Seattle, WA","Reviewed May 26, 2023",5.0,Me and my friend were at Starbucks and my card...,['No Images']
4,Tenessa,"Gresham, OR","Reviewed Jan. 22, 2023",5.0,I’m on this kick of drinking 5 cups of warm wa...,['https://media.consumeraffairs.com/files/cach...


In [17]:
import pandas as pd
import numpy as np
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Replace this with your actual CSV path
csv_path = r"C:\Users\Anjal\OneDrive\Desktop\starbucks_reviews\archive (3)\reviews_data.csv"

# Load data
df = pd.read_csv(csv_path)

# Drop rows with missing review or rating
df = df.dropna(subset=['Review', 'Rating'])

# Create sentiment column: positive if rating >= 4, else negative
df['sentiment'] = df['Rating'].apply(lambda x: 'positive' if x >= 4 else 'negative')

# (Optional) Basic preprocessing function
def preprocess_text(text):
    text = text.lower()
    # Add more preprocessing here if needed (e.g., remove punctuation)
    return text

df['cleaned_review'] = df['Review'].apply(preprocess_text)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42
)

# Vectorize text
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save model and vectorizer
os.makedirs('model', exist_ok=True)
joblib.dump(model, 'model/sentiment_model.pkl')
joblib.dump(vectorizer, 'model/tfidf_vectorizer.pkl')

print("✅ Model and vectorizer saved successfully!")


Accuracy: 0.7943262411347518
              precision    recall  f1-score   support

    negative       0.79      1.00      0.88       111
    positive       1.00      0.03      0.06        30

    accuracy                           0.79       141
   macro avg       0.90      0.52      0.47       141
weighted avg       0.84      0.79      0.71       141

✅ Model and vectorizer saved successfully!


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42
)


In [19]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [24]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')


In [20]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)


LogisticRegression(max_iter=1000)

In [22]:
y_pred = model.predict(X_test_vec)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.7943262411347518

Classification Report:
               precision    recall  f1-score   support

    negative       0.79      1.00      0.88       111
    positive       1.00      0.03      0.06        30

    accuracy                           0.79       141
   macro avg       0.90      0.52      0.47       141
weighted avg       0.84      0.79      0.71       141



In [23]:
os.makedirs('../model', exist_ok=True)
joblib.dump(model, '../model/sentiment_model.pkl')
joblib.dump(vectorizer, '../model/tfidf_vectorizer.pkl')

print("✅ Model and vectorizer saved successfully!")


✅ Model and vectorizer saved successfully!


In [28]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True